In [1]:
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [ ]:
#GROQ_API_KEY="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"

In [2]:
import os

# Set the environment variable
os.environ["GROQ_API_KEY"] = "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"

# Access it
groq_api_key = os.environ["GROQ_API_KEY"]
print(groq_api_key)

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


In [ ]:
#!pip install yfinance
#! pip install langchain_groq
#! pip install  langchain-core 0.3.5
#!pip install openai
!pip install pydantic

In [3]:
import yfinance as yf

yf.__version__

'0.2.43'

In [4]:
from langchain_groq import ChatGroq

llama3 = ChatGroq(api_key=groq_api_key, model="llama-3.1-70b-versatile")

llama3

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C42C85F150>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C42CA8ABD0>, model_name='llama-3.1-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
llama3.invoke("Hello, How are you?")

AIMessage(content="I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help you with any questions or topics you'd like to discuss. How about you? How's your day going?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 41, 'total_tokens': 92, 'completion_time': 0.204, 'prompt_time': 0.010333805, 'queue_time': 0.101351944, 'total_time': 0.214333805}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs': None}, id='run-b9633a41-cac4-473a-90da-c4992861f984-0', usage_metadata={'input_tokens': 41, 'output_tokens': 51, 'total_tokens': 92})

In [6]:
from langchain_core.tools import tool, StructuredTool
import yfinance as yf

@tool
def company_address(ticker: str)-> str:
    """
    Returns company address for input ticker.    
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return " ".join([info[key] for key in ["address1", "city", "state", "zip", "country"]])

@tool
def fulltime_employees(ticker: str)-> int:
    """
    Returns fulltime employees count for input ticker.
    e.g. fulltime_employees: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["fullTimeEmployees"]

@tool
def last_close_price(ticker: str)-> float:
    """
    Returns last close price for input ticker.
    e.g. last_close_price: MSFT
    Returns latest close price for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["previousClose"]

@tool
def EBITDA(ticker: str)-> float:
    """
    Returns EBITDA for input ticker.
    e.g. EBITDA: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["ebitda"]

@tool
def total_debt(ticker: str)-> float:
    """
    Returns total debt for input ticker.
    e.g. total_debt: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["totalDebt"]

@tool
def total_revenue(ticker: str)-> float:
    """
    Returns total revenue for input ticker.
    e.g. total_revenue: MSFT
    Returns total revenue for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["totalRevenue"]

@tool
def debt_to_equity_ratio(ticker: str)-> float:
    """
    Returns debt to equity ratio for input ticker.
    e.g. debt_to_equity_ratio: AAPL
    Returns debt to equity ratio for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["debtToEquity"]

tools = [
    company_address,
    fulltime_employees,
    last_close_price,
    EBITDA,
    total_debt,
    total_revenue,
    debt_to_equity_ratio
]

In [7]:
llama3_with_tools = llama3.bind_tools(tools, tool_choice="auto") 


In [8]:
ai_msg = llama3_with_tools.invoke("Whatzzzz up?")

print(ai_msg.content)
print(ai_msg.tool_calls)

I'm ready to help with stock information. Which ticker would you like to know more about?
[]


In [9]:
ai_msg = llama3_with_tools.invoke("Whats EBITDA of Microsoft?")

print(ai_msg.content)
print(ai_msg.tool_calls)


[{'name': 'EBITDA', 'args': {'ticker': 'MSFT'}, 'id': 'call_2xf3', 'type': 'tool_call'}]


In [10]:
EBITDA.invoke(ai_msg.tool_calls[0]["args"])

129433001984

In [11]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

class AgentState(TypedDict): ## Agent's current state, it can be history of messsages and other attributes you want to maintain
    messages: Annotated[list[AnyMessage], operator.add]   ## {'messages': []}
        
class ReActAgent:
    
    def __init__(self, model, tools, system=""):
        self.system = system
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools, tool_choice="auto")
        
        graph = StateGraph(AgentState)  ## Current Agent State will be available at each node of Graph.
        graph.add_node("llama3", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llama3",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llama3")
        graph.set_entry_point("llama3")
        self.graph = graph.compile()
        
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]} ## AIMessage

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling Tool: {t}")
            if not t['name'] in self.tools:  # check for bad tool name from LLM
                print(f"\n Tool: {t} does not exist.")
                result = "Incorrect Tool Name, Please Retry and Select tool from List of Available tools."  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Tools Execution Complete. Back to the model!")
        return {'messages': results} ## [ToolMessage, ToolMessage, ...]

In [12]:
prompt = """
You are a smart AI finance assistant. Use the list of available tools to answer questions if needed.
You are allowed to make multiple calls (either together or in sequence).
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

agent = ReActAgent(llama3, tools, system=prompt)

In [13]:
!pip install  pygraphviz

  Using cached pygraphviz-1.13.tar.gz (104 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [54 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-311\pygraphviz
  copying pygraphviz\agraph.py -> build\lib.win-amd64-cpython-311\pygraphviz
  copying pygraphviz\graphviz.py -> build\lib.win-amd64-cpython-311\pygraphviz
  copying pygraphviz\scraper.py -> build\lib.win-amd64-cpython-311\pygraphviz
  copying pygraphviz\testing.py -> build\lib.win-amd64-cpython-311\pygraphviz
  copying pygraphviz\__init__.py -> build\lib.win-amd64-cpython-311\pygraphviz
  creating build\lib.win-amd64-cpython-311\pygraphviz\tests
  copying pygraphviz\tests\test_attribute_defaults.py -> build\lib.win-amd64-cpython-311\pygraphviz\tests
  copying pygraphviz\tests\test_clear.py -> build\lib.win-amd64-cpython-311\pygraphviz\tests
  copying pygraphviz\tests\test_close.py -> build\lib.win-amd64-cpython-311\pygraphviz\tests
  copying pygraphviz\tests\test_drawing.py -> build\lib.win-amd6

In [14]:
messages = [HumanMessage(content="What is total revenue of nvidia?")]

result = agent.graph.invoke({"messages": messages})

Calling Tool: {'name': 'total_revenue', 'args': {'ticker': 'NVDA'}, 'id': 'call_pfy2', 'type': 'tool_call'}
Tools Execution Complete. Back to the model!


In [17]:
result['messages'][-1].content

'The total revenue of NVIDIA is $96.31 billion.'

In [18]:
messages = [HumanMessage(content="Compare total revenue of Apple and Nvidia.")]

result = agent.graph.invoke({"messages": messages})

Calling Tool: {'name': 'total_revenue', 'args': {'ticker': 'AAPL'}, 'id': 'call_81xb', 'type': 'tool_call'}
Calling Tool: {'name': 'total_revenue', 'args': {'ticker': 'NVDA'}, 'id': 'call_9pw4', 'type': 'tool_call'}
Tools Execution Complete. Back to the model!


In [19]:
result['messages'][-1].content

'The total revenue of Apple (AAPL) is $385,603,010,560, and the total revenue of Nvidia (NVDA) is $96,307,003,392.'

In [66]:
result['messages'][-1].content

'The total revenue of Apple is 385.6 billion USD and the total revenue of Nvidia is 96.3 billion USD.'

In [20]:
messages = [HumanMessage(content="What is last closing price of nvidia?")]

result = agent.graph.invoke({"messages": messages})

Calling Tool: {'name': 'last_close_price', 'args': {'ticker': 'NVDA'}, 'id': 'call_2tvt', 'type': 'tool_call'}
Tools Execution Complete. Back to the model!


In [21]:
result['messages'][-1].content

'The last closing price of NVIDIA is $116.0.'

In [45]:
!pip install -U -q huggingface_hub

In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
from huggingface_hub import HfApi

HF_USERNAME = "biswadas"
HF_REPOSITORY_NAME = "finance_Agent"

api = HfApi()

api.upload_folder(
    folder_path = "Finance",
    path_in_repo = "FinAgent",
    repo_id = f"{HF_USERNAME}/{HF_REPOSITORY_NAME}",
    repo_type="space"
)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/spaces/biswadas/finance_Agent/commit/2fef284aed3eed58019d7fa8f7fe2b3ccc53b738', commit_message='Upload folder using huggingface_hub', commit_description='', oid='2fef284aed3eed58019d7fa8f7fe2b3ccc53b738', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/biswadas/finance_Agent', endpoint='https://huggingface.co', repo_type='space', repo_id='biswadas/finance_Agent'), pr_revision=None, pr_num=None)

In [32]:
from openai import AzureOpenAI
import json
with open('config1.json', 'r') as az_creds:
    data = az_creds.read()
creds = json.loads(data)
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)
deployment_name = creds["CHATGPT_MODEL"]

In [33]:
import streamlit as st
import openai

# Set up the OpenAI API key (you can load from environment variables or a config file)
#openai.api_key = "YOUR_OPENAI_API_KEY"
api_key=creds["AZURE_OPENAI_KEY"]
# Streamlit app title and description
st.title("Q&A Bot with Streamlit")
st.write("Ask any question and get an answer from the Q&A bot!")

# Function to generate the answer from OpenAI's GPT model
def generate_answer(question):
    try:
        response = openai.Completion.create(
            engine="gpt4-o-mini",  # You can use other engines like "gpt-4" if available
            prompt=f"Q: {question}\nA:",
            max_tokens=150,
            n=1,
            stop=None,
            temperature=0.7,
        )
        answer = response.choices[0].text.strip()
        return answer
    except Exception as e:
        return f"Error: {str(e)}"

# Text input for user's question
question = st.text_input("Enter your question:")

# Button to submit the question
if st.button("Ask"):
    if question:
        with st.spinner("Thinking..."):
            answer = generate_answer(question)
            st.success(answer)
    else:
        st.warning("Please enter a question!")

# Footer
st.write("---")
st.write("Powered by OpenAI GPT and Streamlit.")
